In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = 999

In [2]:
from sklearn.preprocessing import LabelEncoder
train = pd.read_csv("mertrain.csv")
test = pd.read_csv("mertest.csv")

# process columns, apply LabelEncoder to categorical features
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))


Shape train: (4209, 378)
Shape test: (4209, 377)


In [3]:
y = train['y'].values
y_mean = np.mean(y)
id_test = test['ID'].values
X = train.drop(['y','ID'], axis=1).values
X_test= test.drop(['ID'], axis=1).values

In [4]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5, shuffle=True, random_state=0)
kf.get_n_splits(X)

5

In [5]:
for train_index, test_index in kf.split(X):
    X_train, y_train=X[train_index], y[train_index]
    X_valid, y_valid=X[test_index], y[test_index]


In [6]:
forest= RandomForestRegressor(n_estimators=100, random_state=0, max_depth=5)
forest.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [7]:
from sklearn.metrics import r2_score
print("R^2 valid: {: }".format(r2_score(y_valid, forest.predict(X_valid))))
print("R^2 train: {: }".format(r2_score(y_train, forest.predict(X_train))))

R^2 valid:  0.5911715362591088
R^2 train:  0.6025906693231765


In [8]:
y_pred_forest=forest.predict(X_test)

In [11]:
sub=pd.DataFrame()
sub['ID']=id_test
sub["y"]=y_pred_forest
sub.to_csv("forest_maxdepth.csv", index=False)